# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from tensorflow.keras.backend import clear_session

from matplotlib import rcParams, pyplot as plt

import numpy as np
import pandas as pd
import random as rn
import re

from pathlib import Path
import warnings

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

# 학습데이터 로드

In [4]:
seed = 2020
# np.random.seed(seed)
# rn.seed(seed)
# tf.random.set_seed(seed)
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
#                              inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [5]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5

In [6]:
algorithm_name = 'transformer'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


transformer_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
transformer_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
transformer_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
transformer_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


transformer_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
transformer_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
transformer_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
transformer_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


transformer_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
transformer_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
transformer_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
transformer_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

## Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


# 모델 학습

In [15]:
vocab_size = 30000
maxlen = 500
embed_dim = 128
num_heads = 8  # Number of attention heads
padding_type='post'

In [16]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [17]:
def get_model():
    ff_dim = 64  # Hidden layer size in feed forward network inside transformer

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(n_class, activation="softmax")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.001))
    return model

In [18]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

transformer_oof_preds = []
transformer_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=maxlen)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=maxlen)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    transformer_oof_pred = np.zeros((trn.shape[0], n_class))
    transformer_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=32,
            callbacks=[es])
        
        transformer_oof_pred[i_val, :] = clf.predict(trn[i_val])
        transformer_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    transformer_oof_preds.append(transformer_oof_pred)
    transformer_test_preds.append(transformer_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
1372/1372 [==============================] - 124s 91ms/step - loss: 1.3079 - val_loss: 0.9779
Epoch 2/200
1372/1372 [==============================] - 123s 90ms/step - loss: 0.8065 - val_loss: 0.7319
Epoch 3/200
1372/1372 [==============================] - 124s 90ms/step - loss: 0.5109 - val_loss: 0.7323
Epoch 4/200
1372/1372 [==============================] - 126s 92ms/step - loss: 0.3523 - val_loss: 0.8557
Epoch 5/200
1372/1372 [==============================] - 126s 92ms/step - loss: 0.2729 - val_loss: 1.0067
Epoch 00005: early stopping
traing model for CV #2
Epoch 1/200
1372/1372 [==============================] - 127s 92ms/step - loss: 1.3197 - val_loss: 1.0526
Epoch 2/200
1372/1372 [==============================] - 127s 92ms/step - loss: 0.8819 - val_loss: 0.9184
Epoch 3/200
1372/1372 [==============================] - 127s 92ms/step - loss: 0.5883 - val_loss: 0.7622
Epoch 4/200
1372/1372 [===================

1372/1372 [==============================] - 125s 91ms/step - loss: 0.8458 - val_loss: 0.7780
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.5156 - val_loss: 0.7713
Epoch 4/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.3418 - val_loss: 0.9017
Epoch 5/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2630 - val_loss: 1.0048
Epoch 6/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2112 - val_loss: 1.2188
Epoch 00006: early stopping
traing model for CV #5
Epoch 1/200
1372/1372 [==============================] - 124s 91ms/step - loss: 1.2909 - val_loss: 1.0352
Epoch 2/200
1372/1372 [==============================] - 123s 90ms/step - loss: 0.8145 - val_loss: 0.7957
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.5062 - val_loss: 0.7875
Epoch 4/200
1372/1372 [==============================] - 124s 90ms/step - loss: 0.3520 - val_loss: 0.9214
Epoch 5

Epoch 00006: early stopping
traing model for CV #3
Epoch 1/200
1372/1372 [==============================] - 126s 92ms/step - loss: 1.2394 - val_loss: 0.9326
Epoch 2/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.6977 - val_loss: 0.7156
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.4304 - val_loss: 0.7853
Epoch 4/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.3051 - val_loss: 0.9144
Epoch 5/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.2425 - val_loss: 1.0594
Epoch 00005: early stopping
traing model for CV #4
Epoch 1/200
1372/1372 [==============================] - 126s 92ms/step - loss: 1.3167 - val_loss: 1.1634
Epoch 2/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.7873 - val_loss: 0.7476
Epoch 3/200
1372/1372 [==============================] - 125s 91ms/step - loss: 0.4831 - val_loss: 0.7703
Epoch 4/200
1372/1372 [===========================

In [19]:
for i,j in enumerate(transformer_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accruacy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7455
ver1 logloss =  72.5195
ver2 logloss =   0.7689
ver2 logloss =  72.2426
ver3 logloss =   0.7867
ver3 logloss =  71.1948
ver4 logloss =   0.7480
ver4 logloss =  72.9113


# 제출 파일 및 기타 파일 생성

In [21]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = transformer_test_preds[0]
sub.to_csv(transformer_submission_ver1_file)

# Ver2
sub[sub.columns] = transformer_test_preds[1]
sub.to_csv(transformer_submission_ver2_file)

# Ver3
sub[sub.columns] = transformer_test_preds[2]
sub.to_csv(transformer_submission_ver3_file)
           
# Ver4
sub[sub.columns] = transformer_test_preds[3]
sub.to_csv(transformer_submission_ver4_file)

In [22]:
# transformer_oof_pred 파일 생성

# Ver1
np.savetxt(transformer_oof_pred_ver1_file, transformer_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(transformer_oof_pred_ver2_file, transformer_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(transformer_oof_pred_ver3_file, transformer_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(transformer_oof_pred_ver4_file, transformer_oof_preds[3],fmt='%.18f', delimiter=',')

In [23]:
# transformer_test_pred 파일 생성

# Ver1
np.savetxt(transformer_test_pred_ver1_file, transformer_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(transformer_test_pred_ver2_file, transformer_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(transformer_test_pred_ver3_file, transformer_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(transformer_test_pred_ver4_file, transformer_test_preds[3],fmt='%.18f', delimiter=',')